# ニューラルネットワーク自作

ニューラルネットワークをtensorFlowなどのライブラリを使わずに実装する。

In [39]:
import numpy as np
from scipy.special import expit
from typing import List
from tqdm import tqdm

In [9]:
class NeuralNetwork:
    
    # ニューラルネットワークの初期化
    def __init__(self,
                 *,
                 input_nodes: int,
                 hidden_nodes: int,
                 output_nodes: int,
                 learning_rate: float):
        
        # 入力層・隠れ層・出力層のノード数を設定
        self._input_nodes = input_nodes
        self._hidden_nodes = hidden_nodes
        self._output_nodes = output_nodes
        
        # 学習率を設定
        self._learning_rate = learning_rate
        
        # 重み行列のランダム初期化
        # 平均: 0
        # 標準偏差: リンクの数の平方根の逆数
        
        ## 入力層 <-> 隠れ層
        self.weight_input_hidden = np.random.normal(0.0,
                                                    pow(self._hidden_nodes, -0.5),
                                                    (self._hidden_nodes, self._input_nodes))
        
        ## 隠れ層 <-> 出力層
        self.weight_hidden_output = np.random.normal(0.0,
                                                     pow(self._output_nodes, -0.5),
                                                     (self._output_nodes, self._hidden_nodes))
        
        pass
    
    # ニューラルネットワークの学習
    def train(self,
              *,
              inputs_list: List,
              target_list: List):
        
        # トレーニングデータの真の出力リストを行列に変換
        targets = np.array(target_list, ndmin=2).T
        
        # トレーニングデータの出力リスト
        inputs, hiddens, outputs = self.query(inputs_list=inputs_list)
        
        # 出力誤差(目標出力 - 最終出力)
        output_errors = targets - outputs
        
        # 誤差逆伝搬行列
        # E_hidden = W_hidden_output^T ・ E_output
        hidden_errors = np.dot(self.weight_hidden_output.T, output_errors)
        
        # 隠れ層と出力層の間の重みを更新
        self.weight_hidden_output += self._learning_rate * \
                                     np.dot(output_errors * outputs * (1 - outputs), np.transpose(hiddens))
        
        # 入力層と隠れ層の間の重みを更新
        self.weight_input_hidden += self._learning_rate * \
                                    np.dot(hidden_errors * hiddens * (1 - hiddens), np.transpose(inputs))
        
        pass
          
    # ニューラルネットワークの紹介。隠れ層と出力層を返す
    def query(self,
              *,
              inputs_list: List):
        
        # 入力リストを行列に変換。次元数は普通の行列なので2としている
        inputs = np.array(inputs_list, ndmin=2).T
        
        # 隠れ層に入ってくる信号
        # X_hidden = W_input_hidden ・ I
        hiddens = np.dot(self.weight_input_hidden, inputs)
        
        # 活性化関数をかませる
        hiddens = expit(hiddens)
        
        # 出力層に入ってくる信号
        # O = W_hidden_output ・ X_hidden
        outputs = np.dot(self.weight_hidden_output, hiddens)
        
        # 活性化関数をかませる
        outputs = expit(outputs)
        
        return inputs, hiddens, outputs
        
    # ニューラルネットワークの予測    
    def predict(self,
                *,
                inputs_list: List):
        
        return self.query(inputs_list=inputs_list)[2]

MNISTデータセットの取得だけtflearnを用いる。

In [7]:
import tflearn.datasets.mnist as mnist

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [8]:
trainX, trainY, testX, testY = mnist.load_data(one_hot=True)

Extracting mnist/train-images-idx3-ubyte.gz


Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


ニューラルネットワークインスタンスを初期化する。

In [10]:
neural_network = NeuralNetwork(input_nodes=784,
                               hidden_nodes=28,
                               output_nodes=10,
                               learning_rate=0.1)

学習モデルをトレーニングさせる

In [16]:
for _ in tqdm(range(20)):
    for x, y in zip(trainX, trainY):
        neural_network.train(inputs_list=x,
                             target_list=y)

学習済みモデルに対して予測を行う

In [34]:
output = neural_network.predict(inputs_list=testX[39])
print(output[0])
print(output[0].argmax())
print(type(output))

[  2.52832712e-08   9.93375848e-01   9.44112400e-05   3.95555632e-03
   4.00831721e-07   1.49482523e-05   1.01549768e-03   4.63353562e-06
   3.81215028e-03   2.43171791e-06]
1
<class 'numpy.ndarray'>


In [35]:
print(testY[39])
print(testY[39].argmax())

[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
1


In [37]:
right_count = 0
for x, y in zip(testX, testY):
    output = neural_network.predict(inputs_list=x)
    if output[0].argmax() == y.argmax():
        right_count += 1
        
accuracy = right_count / len(testY)
accuracy

0.9428